# TESTING HOLES

In [13]:
import numpy as np
import rasterio
from scipy.ndimage import gaussian_filter
from scipy.ndimage import median_filter

In [7]:
test_tif = "C:/Geomatics/shady_amsterdam/Calculating shade/archive/output/25GN2/CHM_25GN2_12.TIF"

In [20]:
def median_filter(input_file, output_file, nodata_value=-9999, size=3):
    """
    Apply a median filter to a CHM, handling NoData values.

    input:
    - input_file (str): Path to the input CHM GeoTIFF file.
    - output_file (str): Path to the output GeoTIFF file with the smoothed data.
    - nodata_value (float): Value representing NoData in the input raster.
    - size (int): Size of the median filter. It defines the footprint of the filter.

    output:
    - None: The function saves the filtered CHM as a new GeoTIFF file.
    """
    # Load the CHM data
    with rasterio.open(input_file) as src:
        chm = src.read(1)  # Read the first band
        meta = src.meta  # Save metadata for the output file

    # Create a mask for valid data
    valid_mask = chm != nodata_value

    # Pad the data with nodata_value
    pad_width = size // 2
    padded_chm = np.pad(chm, pad_width, mode='constant', constant_values=nodata_value)

    # Apply median filter to padded data
    filtered_padded = median_filter(padded_chm.astype(np.float32), size=size)

    # Remove padding
    smoothed_chm = filtered_padded[pad_width:-pad_width, pad_width:-pad_width]

    # Only keep valid data in smoothed result
    smoothed_chm[~valid_mask] = nodata_value

    # Update metadata for the output file
    out_meta = meta.copy()
    out_meta.update({
        "driver": "GTiff",
        "dtype": 'float32',
        "nodata": nodata_value,
    })

    # Save the smoothed CHM
    with rasterio.open(output_file, 'w', **out_meta) as dst:
        dst.write(smoothed_chm, 1)

    print(f"Median filtered CHM saved to {output_file}")




In [21]:
input_file = test_tif  
output_file = ("C:/Geomatics/shady_amsterdam/Calculating shade/archive/output/output_hole/CHM_25GN2_12.TIF")
size = 3  # Size of the median filter

apply_median_filter_with_nodata_handling(input_file, output_file, size=size)

Median filtered CHM saved to C:/Geomatics/shady_amsterdam/Calculating shade/archive/output/output_hole/smoothed_CHM_25GN2_12.TIF


now use as input for chm, dsm creation and shade map calc

In [38]:
import os
import glob
import re
import geopandas as gpd
import numpy as np
import rasterio
from shapely.geometry import mapping
from rasterio.features import geometry_mask
from scipy.interpolate import NearestNDInterpolator
from functions import write_output
import tqdm
import startinpy
import time
import scipy.ndimage as ndimage
from rasterio import Affine

def get_bbox(raster_paths):
    bboxes = []
    for raster_path in raster_paths:
        with rasterio.open(raster_path) as src:
            bbox = src.bounds
            bboxes.append(bbox)
    left = max([bbox.left for bbox in bboxes])
    bottom = max([bbox.bottom for bbox in bboxes])
    right = min([bbox.right for bbox in bboxes])
    top = min([bbox.top for bbox in bboxes])
    return rasterio.coords.BoundingBox(left, bottom, right, top)

def crop_raster(raster_path, bbox, no_data=-9999, file_number=None, tile=None):
    with rasterio.open(raster_path) as src:
        window = src.window(bbox.left, bbox.bottom, bbox.right, bbox.top)

        # Read the data for the specified window
        cropped_data = src.read(window=window)

        # Ensure window attributes are integers (sometimes bbox in float)
        row_off = int(window.row_off)
        col_off = int(window.col_off)
        height = int(window.height)
        width = int(window.width)

        # Replace no-data values in the data
        if src.nodata is not None:
            cropped_data[cropped_data == src.nodata] = no_data

        # Check for specific tile conditions for cropping
        if tile == "25DN2" and file_number in ["25", "24"]:
            cropped_data = cropped_data[:, :2150, :]  

        return cropped_data, src.window_transform(window), src.crs
    
def extract_center_cells(cropped_data, no_data=-9999):
    # Get coordinates of center cells
    cropped_data = cropped_data[0, :, :]

    # Get the indices of the rows and columns
    rows, cols = np.indices(cropped_data.shape)

    # Identify corner coordinates
    corners = {
        "top_left": (0, 0),
        "top_right": (0, cropped_data.shape[1] - 1),
        "bottom_left": (cropped_data.shape[0] - 1, 0),
        "bottom_right": (cropped_data.shape[0] - 1, cropped_data.shape[1] - 1)
    }
    
    # Mask for valid center cells (non-no_data)
    valid_center_cells = (cropped_data != no_data) 

    # Extract x, y, z values for valid cells
    x_valid = cols[valid_center_cells]
    y_valid = rows[valid_center_cells]
    z_valid = cropped_data[valid_center_cells]

    # Create interpolator from valid points
    interpolator = NearestNDInterpolator(list(zip(x_valid, y_valid)), z_valid)

    # Check each corner for no data and interpolate if necessary
    for corner_name, (row, col) in corners.items():
        if cropped_data[row, col] == no_data:
            # Interpolate the nearest valid value
            cropped_data[row, col] = interpolator((col, row))

    # Extract non-no_data and center cells again after filling corners
    valid_center_cells = (cropped_data != no_data)

    # Extract final x, y, z values after filling corners
    x_filled = cols[valid_center_cells]
    y_filled = rows[valid_center_cells]
    z_filled = cropped_data[valid_center_cells]

    # Prepare final list of [x, y, z]
    xyz_filled = []
    for x_i, y_i, z_i in zip(x_filled, y_filled, z_filled):
        xyz_filled.append([x_i, y_i, z_i])

    return xyz_filled


def fill_raster(cropped_data, nodata_value, transform, speed_up=False):
    # creating delaunay
    points = extract_center_cells(cropped_data, no_data=nodata_value)
    dt = startinpy.DT()
    dt.insert(points, "BBox")
    print("dt")

    # now interpolation
    new_data = np.copy(cropped_data)
    
    # for interpolation, grid of all column and row positions, excluding the first and last rows/cols
    cols, rows = np.meshgrid(
        np.arange(1, cropped_data.shape[2] - 1),
        np.arange(1, cropped_data.shape[1] - 1)

    )

    # flatten the grid to get a list of all (col, row) locations
    locs = np.column_stack((cols.ravel(), rows.ravel()))
    
    # handle cases with large nodata areas if speed_up is true
    large_nodata_region = False
    if speed_up:
        nodata_mask = (cropped_data == nodata_value)
    
        # Find connected nodata regions
        labeled_array, num_features = ndimage.label(nodata_mask)

        # Check if any region meets the minimum size criteria (1100x600 or 600x1100)
        for region_idx in range(1, num_features + 1):
            region_slice = ndimage.find_objects(labeled_array == region_idx)[0]
            region_height = region_slice[1].stop - region_slice[1].start
            region_width = region_slice[2].stop - region_slice[2].start
    
            # Check for a region that meets the size condition in either orientation
            if (region_height >= 1100 and region_width >= 600) or (region_height >= 600 and region_width >= 1100):
                large_nodata_region = True
                break      
                
    # laplace interpolation
    if speed_up and large_nodata_region:
        interpolated_values = dt.interpolate({"method": "TIN"}, locs)
    else:
        interpolated_values = dt.interpolate({"method": "Laplace"}, locs)

    # reshape interpolated grid back to og
    interpolated_grid = np.reshape(interpolated_values, (cropped_data.shape[1] - 2, cropped_data.shape[2] - 2))

    # fill new_data with interpolated values
    new_data[0, 1:-1, 1:-1] = interpolated_grid
    new_data = np.where(np.isnan(new_data), nodata_value, new_data)
    
    new_transform = transform * Affine.translation(1, 1)

    return new_data[0, 1:-1, 1:-1], new_transform

def chm_finish(chm_array, dtm_array, transform):
    result_array = chm_array[0, 1:-1, 1:-1] - dtm_array
    result_array[(result_array < 2) | (result_array > 40)] = 0
    result_array[np.isnan(result_array)] = 0
    
    new_transform = transform * Affine.translation(1, 1)

    return result_array, new_transform

def replace_buildings(filled_dtm, dsm_buildings, buildings_geometries, transform, nodata_value=-9999):

    building_mask = geometry_mask(buildings_geometries, transform=transform, invert=False, out_shape=filled_dtm.shape)

    # Apply the mask to the filled DTM
    final_dtm = np.where((building_mask), filled_dtm, dsm_buildings)

    return final_dtm


def load_buildings(buildings_path, layer):
     buildings_gdf = gpd.read_file(buildings_path, layer = layer)
     return [mapping(geom) for geom in buildings_gdf.geometry]

def extract_tilename(filename):
    match = re.match(r'CHM_(\w+)_\d+\.TIF', filename)
    if match:
        return match.group(1)
    return None


def process_files(chm_files, dtm_path, dsm_path, buildings_path, output_base_folder, nodata_value=-9999, speed_up=False):
    first_chm_filename = os.path.basename(chm_files[0])
    tile = extract_tilename(first_chm_filename)

    if not tile:
        print(f"Skipping {first_chm_filename}, couldn't extract common part.")
        return

    # building geom loaded in once
    building_geometries = load_buildings(buildings_path, layer=tile)

    # output folder for tile
    output_folder = os.path.join(output_base_folder, tile)
    os.makedirs(output_folder, exist_ok=True)
    total_start_time = time.time()


    # Use tqdm to wrap the file list for progress tracking
    for chm_path in tqdm.tqdm(chm_files, desc="Processing Files", unit="file"):
        chm_filename = os.path.basename(chm_path)
        file_number = re.search(r'_(\d+)\.TIF', chm_filename).group(1)
        print(file_number)

        # overlap BBOX dtm, dsm & chm
        raster_paths = [dtm_path, chm_path, dsm_path]
        overlapping_bbox = get_bbox(raster_paths)
        print(f"\nProcessing tile {chm_filename}")
        start_time = time.time()

        # Cropping rasters to bbox
        dtm_cropped, dtm_transform, dtm_crs = crop_raster(dtm_path, overlapping_bbox, no_data=nodata_value, tile=tile, file_number=file_number)
        dsm_cropped, dsm_transform, dsm_crs = crop_raster(dsm_path, overlapping_bbox, no_data=nodata_value, tile=tile, file_number=file_number)
        chm_cropped, chm_transform, _ = crop_raster(chm_path, overlapping_bbox, no_data=nodata_value,  tile=tile, file_number=file_number)
        
       
        print(f"cropped at {(time.time() - start_time):.2f}")
        # no data values fill

        # fill
        filled_dtm, _ = fill_raster(dtm_cropped, nodata_value, dtm_transform, speed_up=speed_up)
        print(f"filled dtm at {(time.time() - start_time):.2f}")

        filled_dsm, new_dsm_transform = fill_raster(dsm_cropped, nodata_value, dsm_transform, speed_up=False)
        print(f"filled dsm at {(time.time() - start_time):.2f}")
        # # norm CHM calculation
        chm_result, new_chm_transform = chm_finish(chm_cropped, filled_dtm, chm_transform)
        #
        # Insert buildings from DSM in DTM
        final_dsm_with_buildings = replace_buildings(filled_dtm, filled_dsm, building_geometries, new_dsm_transform,
                                                     nodata_value)
        print(f"filled buildings at {(time.time() - start_time):.2f}")

        # output file names
        output_dtm_filename = f"DSM_{tile}_{file_number}.tif"
        output_chm_filename = f"CHM_{tile}_{file_number}.tif"

        # Saving final DTM + buildings
        output_dtm_path = os.path.join(output_folder, output_dtm_filename)
        write_output(rasterio.open(dtm_path), final_dsm_with_buildings, new_dsm_transform, output_dtm_path, True)

        # FOR NOW SAVING TO NEW PATH FOR TESTING -> CHANGE THIS WHEN IT WORKS
        output_chm_path = os.path.join(output_folder, output_chm_filename)
        write_output(rasterio.open(chm_path), chm_result, new_chm_transform, output_chm_path, True)

        elapsed_time = time.time() - start_time
        print(f"Processed {chm_filename} in {elapsed_time:.2f} seconds and saved output to {output_dtm_filename}")
    
    total_elapsed_time = time.time() - total_start_time
    print(f"\nAll files processed in {total_elapsed_time:.2f} seconds.")


In [39]:
geotiff_dtm = "../archive/data/DTM_ams.tif"
geotiff_dsm = "../archive/data/DSM_ams.tif"
buildings = "../archive/data/ams_buildings.gpkg"

chm_folder = "D:/Geomatics/CHM/test_interpolation"
output_base_folder = "D:/Geomatics/CHM/output_test"

In [40]:
chm_files = glob.glob(os.path.join(chm_folder, "*.TIF"))
process_files(chm_files, geotiff_dtm, geotiff_dsm, buildings, output_base_folder, nodata_value=-9999, speed_up=True) 

Processing Files:   0%|          | 0/1 [00:00<?, ?file/s]

14

Processing tile CHM_25BZ1_14.TIF
cropped at 1.90
dt
filled dtm at 422.67
dt
filled dsm at 610.81
filled buildings at 610.92


Processing Files: 100%|██████████| 1/1 [10:12<00:00, 612.01s/file]

File written to 'D:/Geomatics/CHM/output_test\25BZ1\DSM_25BZ1_14.tif'
File written to 'D:/Geomatics/CHM/output_test\25BZ1\CHM_25BZ1_14.tif'
Processed CHM_25BZ1_14.TIF in 611.98 seconds and saved output to DSM_25BZ1_14.tif

All files processed in 612.01 seconds.
